In [0]:
from __future__ import print_function


import argparse
import os
import shutil
import time
import random
import torch.nn.functional as F

import argparse
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.nn import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
from torchvision import datasets, transforms
import math

import numpy as np
import torch
from torch.nn import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.functional as F


import torch
import pickle
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import torch.utils.data as Data
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [0]:
class PruningModule(Module):
    def prune_by_percentile(self, q=5.0, **kwargs):
        """
        Note:
             The pruning percentile is based on all layer's parameters concatenated
        Args:
            q (float): percentile in float
            **kwargs: may contain `cuda`
        """
        # Calculate percentile value
        alive_parameters = []
        for name, p in self.named_parameters():
            # We do not prune bias term
            if 'bias' in name or 'mask' in name:
                continue
            tensor = p.data.cpu().numpy()
            alive = tensor[np.nonzero(tensor)] # flattened array of nonzero values
            alive_parameters.append(alive)

        all_alives = np.concatenate(alive_parameters)
        percentile_value = np.percentile(abs(all_alives), q)
        print(f'Pruning with threshold : {percentile_value}')

        # Prune the weights and mask
        # Note that module here is the layer
        # ex) fc1, fc2, fc3
        for name, module in self.named_modules():
            if name in ['fc1', 'fc2', 'fc3']:
                module.prune(threshold=percentile_value)

    def prune_by_std(self, s=0.25):
        """
        Note that `s` is a quality parameter / sensitivity value according to the paper.
        According to Song Han's previous paper (Learning both Weights and Connections for Efficient Neural Networks),
        'The pruning threshold is chosen as a quality parameter multiplied by the standard deviation of a layer’s weights'

        I tried multiple values and empirically, 0.25 matches the paper's compression rate and number of parameters.
        Note : In the paper, the authors used different sensitivity values for different layers.
        """
        for name, module in self.named_modules():
            if name in ['fc1', 'fc2', 'fc3']:
                threshold = np.std(module.weight.data.cpu().numpy()) * s
                print(f'Pruning with threshold : {threshold} for layer {name}')
                module.prune(threshold)


class MaskedLinear(Module):
    r"""Applies a masked linear transformation to the incoming data: :math:`y = (A * M)x + b`

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to False, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Output: :math:`(N, *, out\_features)` where all but the last dimension
          are the same shape as the input.

    Attributes:
        weight: the learnable weights of the module of shape
            (out_features x in_features)
        bias:   the learnable bias of the module of shape (out_features)
        mask: the unlearnable mask for the weight.
            It has the same shape as weight (out_features x in_features)

    """
    def __init__(self, in_features, out_features, bias=True):
        super(MaskedLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        # Initialize the mask with 1
        self.mask = Parameter(torch.ones([out_features, in_features]), requires_grad=False)
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        return F.linear(input, self.weight * self.mask, self.bias)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'

    def prune(self, threshold):
        weight_dev = self.weight.device
        mask_dev = self.mask.device
        # Convert Tensors to numpy and calculate
        tensor = self.weight.data.cpu().numpy()
        mask = self.mask.data.cpu().numpy()
        new_mask = np.where(abs(tensor) < threshold, 0, mask)
        # Apply new weight and mask
        self.weight.data = torch.from_numpy(tensor * new_mask).to(weight_dev)
        self.mask.data = torch.from_numpy(new_mask).to(mask_dev)



In [0]:
def print_model_parameters(model, with_values=False):
    print(f"{'Param name':20} {'Shape':30} {'Type':15}")
    print('-'*70)
    for name, param in model.named_parameters():
        print(f'{name:20} {str(param.shape):30} {str(param.dtype):15}')
        if with_values:
            print(param)


def print_nonzeros(model):
    nonzero = total = 0
    for name, p in model.named_parameters():
        if 'mask' in name:
            continue
        tensor = p.data.cpu().numpy()
        nz_count = np.count_nonzero(tensor)
        total_params = np.prod(tensor.shape)
        nonzero += nz_count
        total += total_params
        print(f'{name:20} | nonzeros = {nz_count:7} / {total_params:7} ({100 * nz_count / total_params:6.2f}%) | total_pruned = {total_params - nz_count :7} | shape = {tensor.shape}')
    print(f'alive: {nonzero}, pruned : {total - nonzero}, total: {total}, Compression rate : {total/nonzero:10.2f}x  ({100 * (total-nonzero) / total:6.2f}% pruned)')



In [0]:
def softmax_by_row(logits, T = 1.0):
    mx = np.max(logits, axis=-1, keepdims=True)
    exp = np.exp((logits - mx)/T)
    denominator = np.sum(exp, axis=-1, keepdims=True)
    return exp/denominator

def classifier_performance(model, train_loader, test_loader):

    output_train_benign = []
    train_label = []
    for num, data in enumerate(train_loader):
        images,labels = data
        image_tensor= images.to(device)
        img_variable = Variable(image_tensor, requires_grad=True)
        output = model.forward(img_variable)

        train_label.append(labels.numpy())
        output_train_benign.append(softmax_by_row(output.data.cpu().numpy(),T = 1))


    train_label = np.concatenate(train_label)
    output_train_benign=np.concatenate(output_train_benign)

    test_label = []
    output_test_benign = []

    for num, data in enumerate(test_loader):
        images,labels = data

        image_tensor= images.to(device)
        img_variable = Variable(image_tensor, requires_grad=True)

        output = model.forward(img_variable)

        test_label.append(labels.numpy())
        output_test_benign.append(softmax_by_row(output.data.cpu().numpy(),T = 1))


    test_label = np.concatenate(test_label)
    output_test_benign=np.concatenate(output_test_benign)


    train_acc1 = np.sum(np.argmax(output_train_benign,axis=1) == train_label.flatten())/len(train_label)
    test_acc1 = np.sum(np.argmax(output_test_benign,axis=1) == test_label.flatten())/len(test_label)

    print('Accuracy: ', (train_acc1, test_acc1))

    return output_train_benign, output_test_benign, train_label, test_label




def inference_via_confidence(confidence_mtx1, confidence_mtx2, label_vec1, label_vec2):
    
    #----------------First step: obtain confidence lists for both training dataset and test dataset--------------
    confidence1 = []
    confidence2 = []
    acc1 = 0
    acc2 = 0
    for num in range(confidence_mtx1.shape[0]):
        confidence1.append(confidence_mtx1[num,label_vec1[num]])
        if np.argmax(confidence_mtx1[num,:]) == label_vec1[num]:
            acc1 += 1
            
    for num in range(confidence_mtx2.shape[0]):
        confidence2.append(confidence_mtx2[num,label_vec2[num]])
        if np.argmax(confidence_mtx2[num,:]) == label_vec2[num]:
            acc2 += 1
    confidence1 = np.array(confidence1)
    confidence2 = np.array(confidence2)
    
    print('model accuracy for training and test-', (acc1/confidence_mtx1.shape[0], acc2/confidence_mtx2.shape[0]) )
    
    
    #sort_confidence = np.sort(confidence1)
    sort_confidence = np.sort(np.concatenate((confidence1, confidence2)))
    max_accuracy = 0.5
    best_precision = 0.5
    best_recall = 0.5
    for num in range(len(sort_confidence)):
        delta = sort_confidence[num]
        ratio1 = np.sum(confidence1>=delta)/confidence_mtx1.shape[0]
        ratio2 = np.sum(confidence2>=delta)/confidence_mtx2.shape[0]
        accuracy_now = 0.5*(ratio1+1-ratio2)
        if accuracy_now > max_accuracy:
            max_accuracy = accuracy_now
            best_precision = ratio1/(ratio1+ratio2)
            best_recall = ratio1
    print('membership inference accuracy is:', max_accuracy)
    return max_accuracy

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
colab='Colab Notebooks'
path = F"/content/gdrive/My Drive/{colab}/bangkok" 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
print("Loading dataset...")
data_set =np.genfromtxt(path,delimiter=',')
print("Dataset loaded.")

Loading dataset...
Dataset loaded.


In [0]:
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, precision_score, recall_score

X = data_set[:,1:].astype(np.float64)
Y = (data_set[:,0]).astype(np.int32)-1


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.45)


train_dataset = Data.TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).long())
test_dataset = Data.TensorDataset(torch.from_numpy(x_test).float(),  torch.from_numpy(y_test).long())


trainloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=True)

In [0]:
batch_size=128
test_batch_size=1000
epochs=50
lr=1e-3
seed=123
log_interval=100


class PurchaseClassifier(PruningModule):
    def __init__(self, mask=False):
        super(PurchaseClassifier, self).__init__()
        linear = MaskedLinear if mask else nn.Linear
        self.fc1=linear(446,512)
        self.fc3=linear(512,256)
        self.fc4=linear(256,128)
        self.tanh = nn.Tanh()
        self.fc5=nn.Linear(128,30)
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc3(x)
        x = self.tanh(x)
        x = self.fc4(x)
        x = self.tanh(x)
        x = self.fc5(x)
        #x = self.softmax(x)
        return x


In [0]:
def train(epochs, model):
    model.train()
    for epoch in range(epochs):

        for batch_idx, (data, target) in enumerate(trainloader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()

            # zero-out all the gradients corresponding to the pruned connections
            for name, p in model.named_parameters():
                if 'mask' in name:
                    continue
                tensor = p.data.cpu().numpy()
                grad_tensor = p.grad.data.cpu().numpy()
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                p.grad.data = torch.from_numpy(grad_tensor).to(device)

            optimizer.step()
            if batch_idx % log_interval == 0:
                done = batch_idx * len(data)
                percentage = 100. * batch_idx / len(trainloader)
                print(f'Train Epoch: {epoch} [{done:5}/{len(trainloader.dataset)} ({percentage:3.0f}%)]  Loss: {loss.item():.6f}')
        test(trainloader, model)
        test(testloader, model)

def test(loader, model):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).sum().item()

        test_loss /= len(loader.dataset)
        accuracy = 100. * correct / len(loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loader.dataset)} ({accuracy:.2f}%)')
    return accuracy

In [0]:
model = PurchaseClassifier(mask=True).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print_model_parameters(model)

# NOTE : `weight_decay` term denotes L2 regularization loss term
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.000)
initial_optimizer_state_dict = optimizer.state_dict()


Param name           Shape                          Type           
----------------------------------------------------------------------
fc1.weight           torch.Size([512, 446])         torch.float32  
fc1.mask             torch.Size([512, 446])         torch.float32  
fc1.bias             torch.Size([512])              torch.float32  
fc3.weight           torch.Size([256, 512])         torch.float32  
fc3.mask             torch.Size([256, 512])         torch.float32  
fc3.bias             torch.Size([256])              torch.float32  
fc4.weight           torch.Size([128, 256])         torch.float32  
fc4.mask             torch.Size([128, 256])         torch.float32  
fc4.bias             torch.Size([128])              torch.float32  
fc5.weight           torch.Size([30, 128])          torch.float32  
fc5.bias             torch.Size([30])               torch.float32  


#Initial Training

In [0]:
# Initial training
print("--- Initial training ---")
train(epochs, model)



print("--- Before pruning ---")
print_nonzeros(model)

output_train, output_test, train_label, test_label = classifier_performance(model, trainloader, testloader)
inference_accuracy=inference_via_confidence(output_train, output_test, train_label, test_label)
print("Maximum Accuracy:",inference_accuracy)


--- Initial training ---
Train Epoch: 0 [    0/2755 (  0%)]  Loss: 3.397657
Test set: Average loss: 0.0177, Accuracy: 1083/2755 (39.31%)
Test set: Average loss: 0.0031, Accuracy: 752/2255 (33.35%)
Train Epoch: 1 [    0/2755 (  0%)]  Loss: 2.152041
Test set: Average loss: 0.0114, Accuracy: 1664/2755 (60.40%)
Test set: Average loss: 0.0021, Accuracy: 1123/2255 (49.80%)
Train Epoch: 2 [    0/2755 (  0%)]  Loss: 1.391172
Test set: Average loss: 0.0081, Accuracy: 2121/2755 (76.99%)
Test set: Average loss: 0.0017, Accuracy: 1369/2255 (60.71%)
Train Epoch: 3 [    0/2755 (  0%)]  Loss: 0.979449
Test set: Average loss: 0.0061, Accuracy: 2299/2755 (83.45%)
Test set: Average loss: 0.0016, Accuracy: 1379/2255 (61.15%)
Train Epoch: 4 [    0/2755 (  0%)]  Loss: 0.789889
Test set: Average loss: 0.0047, Accuracy: 2449/2755 (88.89%)
Test set: Average loss: 0.0015, Accuracy: 1422/2255 (63.06%)
Train Epoch: 5 [    0/2755 (  0%)]  Loss: 0.582505
Test set: Average loss: 0.0036, Accuracy: 2550/2755 (92.56%)

# **Pruning**

In [0]:
# Pruning

print("--- Starting pruning ---")
model.prune_by_std(2)
test_accuracy = test(testloader,model)
train_accuracy = test(trainloader, model)
print("Test Accuracy:", test_accuracy)
print("Train Accuracy:", train_accuracy)
print("--- After pruning ---")
print_nonzeros(model)

output_train, output_test, train_label, test_label = classifier_performance(model, trainloader, testloader)
inference_accuracy=inference_via_confidence(output_train, output_test, train_label, test_label)
print("Maximum Accuracy:",inference_accuracy)

--- Starting pruning ---
Pruning with threshold : 0.06448423862457275 for layer fc1
Pruning with threshold : 0.09618093818426132 for layer fc3
Test set: Average loss: 0.0023, Accuracy: 1390/2255 (61.64%)
Test set: Average loss: 0.0000, Accuracy: 2755/2755 (100.00%)
Test Accuracy: 61.64079822616408
Train Accuracy: 100.0
--- After pruning ---
fc1.weight           | nonzeros =   15657 /  228352 (  6.86%) | total_pruned =  212695 | shape = (512, 446)
fc1.bias             | nonzeros =     512 /     512 (100.00%) | total_pruned =       0 | shape = (512,)
fc3.weight           | nonzeros =    9867 /  131072 (  7.53%) | total_pruned =  121205 | shape = (256, 512)
fc3.bias             | nonzeros =     256 /     256 (100.00%) | total_pruned =       0 | shape = (256,)
fc4.weight           | nonzeros =   32768 /   32768 (100.00%) | total_pruned =       0 | shape = (128, 256)
fc4.bias             | nonzeros =     128 /     128 (100.00%) | total_pruned =       0 | shape = (128,)
fc5.weight           

# **Retraining**

In [0]:
print("--- Retraining ---")
optimizer.load_state_dict(initial_optimizer_state_dict) # Reset the optimizer
train(epochs, model)


print("--- After Retraining ---")
print_nonzeros(model)

output_train, output_test, train_label, test_label = classifier_performance(model, trainloader, testloader)
inference_accuracy=inference_via_confidence(output_train, output_test, train_label, test_label)
print("Maximum Accuracy:",inference_accuracy)

--- Retraining ---
Train Epoch: 0 [    0/2755 (  0%)]  Loss: 0.004827
Test set: Average loss: 0.0000, Accuracy: 2755/2755 (100.00%)
Test set: Average loss: 0.0024, Accuracy: 1371/2255 (60.80%)
Train Epoch: 1 [    0/2755 (  0%)]  Loss: 0.002704
Test set: Average loss: 0.0000, Accuracy: 2755/2755 (100.00%)
Test set: Average loss: 0.0024, Accuracy: 1382/2255 (61.29%)
Train Epoch: 2 [    0/2755 (  0%)]  Loss: 0.002381
Test set: Average loss: 0.0000, Accuracy: 2755/2755 (100.00%)
Test set: Average loss: 0.0025, Accuracy: 1391/2255 (61.69%)
Train Epoch: 3 [    0/2755 (  0%)]  Loss: 0.001675
Test set: Average loss: 0.0000, Accuracy: 2755/2755 (100.00%)
Test set: Average loss: 0.0026, Accuracy: 1390/2255 (61.64%)
Train Epoch: 4 [    0/2755 (  0%)]  Loss: 0.001290
Test set: Average loss: 0.0000, Accuracy: 2755/2755 (100.00%)
Test set: Average loss: 0.0025, Accuracy: 1391/2255 (61.69%)
Train Epoch: 5 [    0/2755 (  0%)]  Loss: 0.001226
Test set: Average loss: 0.0000, Accuracy: 2755/2755 (100.00%